### 广度优先搜索 Breadth First Search

给定图G，以及开始搜索的起始顶点s：

- BFS搜索 所有从s可到达顶点的边（从距离k为1开始）
- 在达到更远的距离k+1的顶点之前，BFS会找到全部距离为k的顶点（宽度优先）
- 可以想象以s为根，构建一棵树的过程，从顶点向下，逐步增加层次
- 广度有限搜索能保证在增加层次之前，添加了所有兄弟节点到树中

对代码的优化需求：

为了跟踪顶点的加入过程，并避免重复顶点，要为顶点增加3个属性：

1. 距离distance：从起始顶点到此顶点的路径长度
2. 前驱顶点predecessor：可反向追溯到起点
3. 颜色color：标识此顶点是尚未发现（白色）、已经发现（灰色）、还是已经完成探索（黑色）
4. 一个队列Queue来对已经发现的顶点进行排列（决定下一个要探索的顶点）


### BSF算法过程
从起始顶点s开始，作为刚发现的顶点，标注为灰色，距离为0，前驱为None，加入队列，接下来是循环迭代过程：

1. 从队首取出一个顶点作为当前顶点；
2. 遍历当前顶点的邻接顶点，如果是尚未发现的白色顶点，则将其颜色改为灰色（已发现），距离增加1，前驱顶点为当前顶点，加入到队列中
3. 遍历完成后，将当前顶点设置为黑色（已探索过），循环回到步骤1的队首取当前节点

### BFS算法分析
主体是两个循环的嵌套
- while循环对每个顶点访问一次，所以复杂度是O(|V|)
- 嵌套在while中的for，由于每条边只有在其起始顶点出队的时候才会被检查一次，而每个顶点最多出队一次，所以边最多被检查1次，复杂度是O(|E|)
- 综合得到时间复杂度O(|V|+|E|)

### BFS的应用：词梯问题 Word Ladder
从一个单词演变到另一个单词，其中过程可以经过多个中间单词

FOOL >> POOL >> POLL >> POLE >> PALE >> SALE >> SAGE

目标是找到**最短**的单词变换序列

采用图来解决这个问题的步骤：

1. 将可能的单词之间的**演变关系**变大为图
2. 采用“广度优先搜索 BFS”，来搜寻从开始单词到结束单词之间的**所有**有效路径
3. 选择其中**最快到达**目标的单词路径

代码实现步骤：

1. 使用包含一个通配符的单词作为顶点的标识key（比如**_OPE**)
2. 将与之匹配的单词放入key中（比如HOPE, POPE等）
3. 这些词之间建立边，表示它们只差一个字母

In [ ]:
def buildGraph(wordFile):
    d = {}
    g = Graph()
    wfile = open(wordFile, 'r')
    for line in wfile:
        word = line[:-1]
        for i in range(len(word)):
            bucket = word[: i] + '_' + word[i + 1:]
            if bucket in d:
                d[bucket].append(word)
            else:
                d[bucket] = [word]
    # 对同一个桶内的节点添加边
    for  bucket in d.keys():
        for word1 in d[bucket]:
            for word2 in d[bucket]:
                if word1 != word2:
                    g.addEdge(word1, word2)
    return g

### 深度优先搜索 Depth First Search

深度优先搜索是沿着树的单支尽量深入向下搜索

如果到无法继续的程度还未找到问题解，就回溯上一层再搜索下一支


### DFS的两种实现算法
1. 一个DFS可用于解决骑士周游问题，其特点是每个顶点仅访问一次
2. 一个DFS算法更为通用，允许顶点被**重复**访问，可作为其它图算法的基础

### 应用：骑士周游问题

在一个国际象棋棋盘上，一个旗子“马”，从一个格子出发，走完棋盘，称为一次周游，要求求出所有周游路线

### 解决思路
将棋盘和走棋步骤构建为图
- 将棋盘格作为顶点
- 按照“马走日”规则的走棋步骤作为连接边
- 建立每一个棋盘格的所有合法走棋步骤能够到达的棋盘格关系图

### 方法一：针对骑士周游

- 如果沿着单支深入搜索到无法继续（所有合法移动都已经被走过了）时，路径长度还没有达到预定值（8*8棋盘为63），那么就清除颜色标记，返回到上一层
- 换一个分支继续深入搜索
- 引入一个栈来记录路径，并实施返回上一层的回溯操作

In [ ]:
def genLegalMoves(x, y, bdSize):
    newMoves = []
    moveOffsets = [(-1, -2), (-1, 2), (-2, -1), (-2, -1), (1, -2), (1, 2), (2, -1), (2,1)] #马可以走的8个方向的路线
    for i in moveOffsets:
        newX = x + i[0]
        newY = y + i[1]
        if legalCoord(newX, bdSize) and legalCoord(newY, bdSize):
            newMoves.append(newX, newY)
    return newMoves

def legaoCoord(x, bdSize):
    if x >= 0 and x < bdSize:
        return True
    else:
        return False

    def knightGraph(bdSize):
    ktGraph = Graph()
    for row in range(bdSize):
        for col in range(bdSize):
            nodeId = posToNodeId(row, col, bdSize)
            newPositions = genLegalMoves(row, col, bdSize)
            for e in newPositions:
                nid = posToNodeId(e[0], e[1], bdSize)
                ktGraph.addEdge(nodeId, nid)
    return ktGraph

def posToNodeId(row, col, bdSize):
    return row * bdSize + col

def knightTour(n, paht, u, limit): #n：层次；path：路径；u：当前顶点；limit：搜索总深度
    u.setColor('gray')
    path.append(u)
    if n < limit: #如果当前搜索还没到达全部棋盘走完
        nbrList = list(u.getConnections()) #获取合法移动清单
        i = 0
        done = False
        while i < len(nbrlist) and not done:
            if nbrList[i].getColor() == 'white': #选择未经过的点深入
                done = knightTour(n+1, path, nbrList[i], limit)
            i = i + 1
        if not done: #深入失败，准备回溯
            path.pop()
            u.setColor('white')
    else:
        done = True
    return done

骑士周游问题的缺点

这个算法是依赖于棋盘大小n的指数时间复杂度$O(n^k)$

通过对nbrList的改造，让骑士以特定方式排列顶点访问次序，可以极大降低搜索时间

将合法移动目标棋格的排序修改为，具有最少合法移动目标的格子优先搜索

In [ ]:
def orderByAvail(n):
    resLilst = []
    for v in n.getConnections():
        if v.getColor() == 'white':
            c = 0
            for w in v.getConnections():
                if w.getColor() == 'while':
                    c = c + 1
            resList.append((c, v))
    resList.sort(key=lambda x: x[0])
    return [y[1] for y in resList]

这种算法是使用先验知识来改进算法，称作为“启发式规则”

### 方法二：通用
一般的深度优先搜索是在图上进行尽量深的搜索，连接尽量多的顶顶啊，必要时可进行分支（创建了树）

有时候深度优先搜索会创建多棵树，称为“深度优先森林”

深度优先搜索同样要用到顶点的“前驱”属性，来构建树或森林

另外要设置“发现时间”和“结束时间”属性
- 发现时间：在第几步访问到这个顶点（设置灰色）
- 结束时间：第几步完成了此定点搜索（设置黑色）

### 通用深度优先搜索算法分析

DFS运行时间包含了两个方面：
- dfs函数中有两个循环，每个都是|v|次，所以时间复杂度是O(|v|)
- dfsvisit函数中的循环则是对当前顶点所连接的顶点进行，而且仅在顶点为白色的情况下才递归调用，所以对每条边来说只会运行一步，时间复杂度是O(|E|)

两者加起来就是和BFS一样的O(|V|+|E|)